<a href="https://colab.research.google.com/github/NthabisengMoela/LANGUAGE-IDENTIFICATION-HACKATHON/blob/main/Nthabiseng_Hackathon_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Loading Data
import pandas as pd
import numpy as np
import nltk
import string
import re
import time

# Data Preprocessing
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.utils import resample
from nltk.tokenize import word_tokenize, TreebankWordTokenizer
from nltk.stem import WordNetLemmatizer

# Model Building
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, cross_val_score

# Model Evaluation
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier
#from scikitplot.metrics import plot_roc, plot_confusion_matrix

# Explore Data Analysis
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from wordcloud import WordCloud, STOPWORDS
from matplotlib.pyplot import rcParams

from sklearn.feature_extraction.text import CountVectorizer

from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score

In [ ]:
train = pd.read_csv('train_set.csv')
test = pd.read_csv('test_set.csv')

In [ ]:
ftrain = train.copy()
ftest = test.copy()

In [ ]:
import re
def  clean_text(df, text_field):
    df[text_field] = df[text_field].str.lower()
    df[text_field] = df[text_field].apply(lambda elem: re.sub(r"(@[A-Za-z0-9]+)|('[0-9]+')|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", elem))  
    return df
test_clean = clean_text(ftest, "text")
train_clean = clean_text(ftrain, "text")

In [ ]:
test_clean.head(20)

,index,text
0,1,mmasepala fa maemo a a kgethegileng a letlelel...
1,2,uzakwaziswa ngokufaneleko nakungafuneka eminye...
2,3,tshivhumbeo tshi fana na ngano dza vhathu
3,4,kube inja nelikati betingevakala kutsi titsini...
4,5,winste op buitelandse valuta
5,6,ke feela dilense te hlakilego ta pono e tee go...
6,7,fn762010101403 am 1495 final gems birthing opt...
7,8,ntjhafatso ya konteraka ya mosebetsi etsa bonn...
8,9,ugems uhlinzeka ngezinzuzo zemithi yezifo ezin...
9,10,so on occasion are statistics misused


In [ ]:
test_clean['text'] = test_clean['text'].replace(to_replace = '[0-9]+', value = '', regex = True)
test_clean.head(20)

,index,text
0,1,mmasepala fa maemo a a kgethegileng a letlelel...
1,2,uzakwaziswa ngokufaneleko nakungafuneka eminye...
2,3,tshivhumbeo tshi fana na ngano dza vhathu
3,4,kube inja nelikati betingevakala kutsi titsini...
4,5,winste op buitelandse valuta
5,6,ke feela dilense te hlakilego ta pono e tee go...
6,7,fn am final gems birthing optionszulutxtfn
7,8,ntjhafatso ya konteraka ya mosebetsi etsa bonn...
8,9,ugems uhlinzeka ngezinzuzo zemithi yezifo ezin...
9,10,so on occasion are statistics misused


In [ ]:
train_clean['text'] = train_clean['text'].replace(to_replace = '[0-9]+', value = '', regex = True)
train_clean.head(20)

,lang_id,text
0,xho,umgaqosiseko wenza amalungiselelo kumaziko axh...
1,xho,idha iya kuba nobulumko bokubeka umsebenzi nap...
2,eng,the province of kwazulunatal department of tra...
3,nso,o netefata gore o ba file dilo ka moka te le d...
4,ven,khomishini ya ndinganyiso ya mbeu yo ewa maana...
5,nso,dinyakiio te ta go dirwa gabedi ka ngwaga di t...
6,tsn,kgetse nngwe le nngwe e e sa faposiwang mo tsh...
7,ven,mbadelo dze dza laelwa dzi do kwama mahatulele...
8,nso,maloko a dikhuduthamaga a ikarabela mongwe le ...
9,tsn,fa le dirisiwa lebone le tshwanetse go bontsha...


In [ ]:
# creating instance of labelencoder
from sklearn.preprocessing import LabelEncoder

labelencoder = LabelEncoder()
# Assigning numerical values and storing in another column
train_clean['lang_num'] = labelencoder.fit_transform(train_clean['lang_id'])
train_clean

,lang_id,text,lang_num
0,xho,umgaqosiseko wenza amalungiselelo kumaziko axh...,9
1,xho,idha iya kuba nobulumko bokubeka umsebenzi nap...,9
2,eng,the province of kwazulunatal department of tra...,1
3,nso,o netefata gore o ba file dilo ka moka te le d...,3
4,ven,khomishini ya ndinganyiso ya mbeu yo ewa maana...,8
...,...,...,...
32995,tsn,popo ya dipolateforomo tse ke go tlisa boetele...,6
32996,sot,modise mosadi na o ntse o sa utlwe hore thaban...,4
32997,eng,closing date for the submission of completed t...,1
32998,xho,nawuphina umntu ofunyenwe enetyala phantsi kwa...,9


# Training the model

In [ ]:
X = train_clean['text']
y = train_clean['lang_num']

In [ ]:
#Vectorize the clean text
vectorizer = TfidfVectorizer(sublinear_tf=True)
X_vectorized = vectorizer.fit_transform(X)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_vectorized, y, test_size=0.2, random_state=42)

In [ ]:
# Classifier - Algorithm - Naive Bayes
# fit the training dataset on the classifier
Naive = naive_bayes.MultinomialNB()
Naive.fit(X_train,y_train)
# predict the labels on validation dataset
predictions_NB = Naive.predict(X_test)

# Use accuracy_score function to get the accuracy
print("Naive Bayes Accuracy Score -> ",accuracy_score(predictions_NB, y_test)*100)
print("F1 Score -> ",f1_score(predictions_NB, y_test, average="macro")*100)

Naive Bayes Accuracy Score ->  99.86363636363636
F1 Score ->  99.86277573330699


In [ ]:
lsvc = LinearSVC()

lsvc.fit(X_train,y_train)
# predict the labels on validation dataset
predictions_lsvc = lsvc.predict(X_test)

# Use accuracy_score function to get the accuracy
print("SVM Accuracy Score -> ",accuracy_score(predictions_lsvc, y_test)*100)
print("F1 Score -> ",f1_score(predictions_lsvc, y_test, average="macro")*100)

SVM Accuracy Score ->  99.7121212121212
F1 Score ->  99.70973770815422


In [ ]:
svc = SVC()

svc.fit(X_train,y_train)
# predict the labels on validation dataset
predictions_svc = svc.predict(X_test)

# Use accuracy_score function to get the accuracy
print("SVM Accuracy Score -> ",accuracy_score(predictions_svc, y_test)*100)
print("F1 Score -> ",f1_score(predictions_lsvc, y_test, average="macro")*100)

SVM Accuracy Score ->  99.57575757575758
F1 Score ->  99.70973770815422


In [ ]:
# Training Logistics Regression model
LR_model = LogisticRegression(solver='lbfgs')
LR_model.fit(X_train, y_train)
y_predict_lr = LR_model.predict(X_test)
print(accuracy_score(y_test, y_predict_lr))
print("F1 Score -> ",f1_score(y_predict_lr, y_test, average="macro")*100)

C:\Users\zoemo\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.9956060606060606
F1 Score ->  99.55467694270325


# Grid Search 

In [ ]:
LinearSVC().get_params()

{'C': 1.0,
 'class_weight': None,
 'dual': True,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'loss': 'squared_hinge',
 'max_iter': 1000,
 'multi_class': 'ovr',
 'penalty': 'l2',
 'random_state': None,
 'tol': 0.0001,
 'verbose': 0}

In [ ]:
param_grid = {'C': [100],#[0.1,1,10,100,1000]
              'max_iter': [3 0], #[10,100]
              'multi_class' : ['ovr'], #['crammer_singer', 'ovr']
              'random_state': [42]} 
grid_LSVC = GridSearchCV(LinearSVC(), param_grid, scoring='f1_weighted', cv=5, n_jobs=-1)
grid_LSVC.fit(X_train, y_train)
y_pred = grid_LSVC.predict(X_test)
print(grid_LSVC.best_params_)

print("LSVM Accuracy Score -> ",accuracy_score(y_pred, y_test)*100)
print("F1 Score -> ",f1_score(y_pred, y_test, average="macro")*100)

{'C': 100, 'max_iter': 10, 'multi_class': 'ovr', 'random_state': 42}
LSVM Accuracy Score ->  99.69696969696969
F1 Score ->  99.69736746704996


C:\Users\zoemo\anaconda3\lib\site-packages\sklearn\svm\_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [ ]:
#Save the model

testx = test_clean['text']
test_vect = vectorizer.transform(testx)

naive_pred = Naive.predict(test_vect)
otest = test_clean.copy()
otest['lang_id'] = naive_pred
otest.head()

,index,text,lang_id
0,1,mmasepala fa maemo a a kgethegileng a letlelel...,tsn
1,2,uzakwaziswa ngokufaneleko nakungafuneka eminye...,nbl
2,3,tshivhumbeo tshi fana na ngano dza vhathu,ven
3,4,kube inja nelikati betingevakala kutsi titsini...,ssw
4,5,winste op buitelandse valuta,afr


In [ ]:
df = otest[['index','lang_id']]
df.head()

,index,lang_id
0,1,tsn
1,2,nbl
2,3,ven
3,4,ssw
4,5,afr


In [ ]:
df.to_csv('base_submission5.csv', index=False)

In [ ]:
Re do

In [ ]:
train = pd.read_csv('train_set.csv')
test = pd.read_csv('test_set.csv')

In [ ]:
ftrain = train.copy()
ftest = test.copy()

In [ ]:
def clean(df,text_field):

    
    df[text_field] = df[text_field].str.replace(r'^.+@[^\.].*\.[a-z]{2,}$',
                                     'emailaddress')

    # Remove unicode strings
    df[text_field] = df[text_field].str.replace(r'(\\u[0-9A-Fa-f]+)', '')
    df[text_field] = df[text_field].str.replace(r'[^\x00-\x7f]', '')

    # Convert any url to URL
    df[text_field] = df[text_field].str.replace('((www\.[^\s]+)|(https?://[^\s]+))',
                                'website')

    # Remove special characters, numbers, punctuation
    df[text_field] = df[text_field].str.replace('[^a-zA-Z#@]+',' ')

    # Remove additional white spaces
    df[text_field] = df[text_field].str.replace(r'\s+', ' ')
    df[text_field] = df[text_field].str.replace(r'^\s+|\s+?$', '')

    # Change words to lower case
    df[text_field] = df[text_field].str.lower()

    # Remove stop words from tweets
    stop_words = set(stopwords.words('english'))

    df[text_field] = df[text_field].apply(lambda x: ' '.join(
        word for word in x.split() if word not in stop_words))

    return df

In [ ]:
ftrain[clean] =  clean(ftrain,text)
ftest[clean] =  clean(ftest,text)

NameError: name 'text' is not defined

In [ ]:
vectorizer = CountVectorizer(analyzer = 'word', 
                             tokenizer = None, 
                             preprocessor = None, 
                             stop_words = None, 
                             max_features = 180000,
                             min_df = 1,
                             ngram_range = (1,2)
                            )

In [ ]:
# Import library for Feature Extraction
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer

# Import library for train test split
from sklearn.model_selection import train_test_split

# Import Machine Learning algorithms
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

In [ ]:
# Building a Pipeline for word vectorization
pipe = Pipeline( [('vect', vectorizer)] )


In [ ]:
X = train_clean['text']
y = train_clean['lang_num']

In [ ]:
# Fitting & transforming the data
train_vect = pipe.fit_transform(X)
test_vect = pipe.transform(test_clean['text'])

In [ ]:
# Splitting the data into train & test sets
X_train, X_test, y_train, y_test = train_test_split(train_vect, y,
                                                    test_size = 0.2, random_state=42)

In [ ]:
# These are the different Classification models we will train our data on
# Creating a list of names so we can print metrics for the entire list at once
names = ['Logistic Regression'
         ,'Nearest Neighbors'
         ,'Linear SVC'
         , 'RBF SVC'
         , 'Linear SVM'
         , 'Decision Tree'
         , 'Random Forest'
         ,  'AdaBoost'
]

In [ ]:
# These are the different Classification models we will train our data on
# Creating a list of names so we can print metrics for the entire list at once
classifiers = [
    LogisticRegression()
    , KNeighborsClassifier(3)
    , SVC(kernel="linear", C=0.025)
    , SVC(gamma=2, C=1)
    , LinearSVC()
    , DecisionTreeClassifier(max_depth=5)
    , RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1)
    , AdaBoostClassifier()
]

In [ ]:
# Setting the number of folds

kfold = StratifiedKFold( n_splits = 150)

In [ ]:
# Linear SVC model
#Set the parameters according to the first run output (C = 0.09)
param_grid2 = {'C': [0.09]}

# Grid Search on the Linear SVC model
gs_sv = GridSearchCV(LinearSVC(), param_grid=[param_grid2], 
                     verbose=1, cv=kfold, n_jobs=-1)

# Fitting the model to our data
gs_sv.fit(X_train,y_train)

# Find the best estimator
gs_sv_best = gs_sv.best_estimator_

# Print the best parameters
print(gs_sv.best_params_)

# Print the best score
print(gs_sv.best_score_)

Fitting 150 folds for each of 1 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  5.1min
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed: 13.1min finished


{'C': 0.09}
0.996628787878788
